In [1]:
 # Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pandas as pd
from splinter import Browser
import time

In [2]:
 # Initialize PyMongo to work with MongoDBs
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

In [3]:
# Keeping pandas from truncating long strings
#source: https://coding-stream-of-consciousness.com/2020/04/17/python-pandas-stop-truncating-strings/
pd.set_option('display.max_colwidth', -1)

C:\Users\Amber\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# URL of page to be scraped
url = 'https://dogtime.com/dog-breeds'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')

In [5]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='list-item')
results

[<div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/afador">Afador</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/affenhuahua">Affenhuahua</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/affenpinscher">Affenpinscher</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/afghan-hound">Afghan Hound</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/airedale-terrier">Airedale Terrier</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/akbash">Akbash</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/akita">Akita</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/akita-chow">Akita Chow</a></div>,
 <div class="list-item"><a

In [6]:
# loop over results to get article data

dog_breed_list = []
dog_links = []
for result in results:
    # scrape the dog types 
    dog_type = result.find('a', class_='list-item-title').text
    dog_breed_list.append(dog_type)
    
    #scrape the link
    link = result.a['href']
    dog_links.append(link)

In [7]:
#zip dog list and dog link lists together
combined_lists = list(zip(dog_breed_list, dog_links))


In [8]:
#make combined_lists into a dataframe
dog_info_df = pd.DataFrame(combined_lists)

In [22]:
#rename columns 
dog_info_df = dog_info_df.rename(columns = {0:"Dog", 1:"Link"})
dog_info_df

,Dog,Link
0,Afador,https://dogtime.com/dog-breeds/afador
1,Affenhuahua,https://dogtime.com/dog-breeds/affenhuahua
2,Affenpinscher,https://dogtime.com/dog-breeds/affenpinscher
3,Afghan Hound,https://dogtime.com/dog-breeds/afghan-hound
4,Airedale Terrier,https://dogtime.com/dog-breeds/airedale-terrier
...,...,...
372,Whoodle,https://dogtime.com/dog-breeds/whoodle
373,Wirehaired Pointing Griffon,https://dogtime.com/dog-breeds/wirehaired-pointing-griffon
374,Xoloitzcuintli,https://dogtime.com/dog-breeds/xoloitzuintli
375,Yorkipoo,https://dogtime.com/dog-breeds/yorkipoo


In [10]:
#read csv of petfinder dogs
petfinder_dogs_df = pd.read_csv('data/petfinder_dogs.csv')
petfinder_dogs_df

,city,state,pet_id,name,breed
0,Andover,MA,48752074,Jade,Redbone Coonhound
1,New York,NY,48752069,Wish,Labrador Retriever
2,TULSA,OK,48752062,Molly,Shiba Inu
3,White Plains,NY,48751967,Brindi,Retriever
4,Wasco,CA,48751842,(FRECKLES) AVAILABLE AUG 24 2020,Greyhound
...,...,...,...,...,...
490,Dallas,TX,48750352,1105763,Pit Bull Terrier
491,Santa Fe,NM,48750344,ZUMA,Mixed Breed
492,Santa Cruz,CA,48750345,MUPPET*,Maltese
493,Santa Fe,NM,48750342,JEREMIAH,Mixed Breed


In [11]:
#get list of dogs from petfinder csv
#source: https://stackoverflow.com/questions/22341271/get-list-from-pandas-dataframe-column
petfinder_dog_list = petfinder_dogs_df['breed'].tolist()

In [12]:
search_list = []
for dog in petfinder_dog_list:
    link = dog_info_df.loc[dog_info_df['Dog']== dog, 'Link' ]
    link = str(link)
    if link != 'Series([], Name: Link, dtype: object)':
        link = link.split(' ')
        link = link[4]
        link = link.split('\n')
        search_list.append(link[0])
        

In [13]:
#removing duplicates
#source: https://www.geeksforgeeks.org/python-set-method/#:~:text=set()%20method%20is%20used,dintinct%20elements%2C%20commonly%20called%20Set.&text=Parameters%20%3A%20Any%20iterable%20sequence%20like,modified%20as%20passed%20as%20argument.
search_list = list(set(search_list)) 
search_list

['https://dogtime.com/dog-breeds/greyhound',
 'https://dogtime.com/dog-breeds/pomeranian',
 'https://dogtime.com/dog-breeds/akita',
 'https://dogtime.com/dog-breeds/bloodhound',
 'https://dogtime.com/dog-breeds/collie',
 'https://dogtime.com/dog-breeds/black-and-tan-coonhound',
 'https://dogtime.com/dog-breeds/pekingese',
 'https://dogtime.com/dog-breeds/rottweiler',
 'https://dogtime.com/dog-breeds/dutch-shepherd',
 'https://dogtime.com/dog-breeds/rhodesian-ridgeback',
 'https://dogtime.com/dog-breeds/siberian-husky',
 'https://dogtime.com/dog-breeds/great-pyrenees',
 'https://dogtime.com/dog-breeds/west-highland-white-terrier',
 'https://dogtime.com/dog-breeds/shetland-sheepdog',
 'https://dogtime.com/dog-breeds/boston-terrier',
 'https://dogtime.com/dog-breeds/great-dane',
 'https://dogtime.com/dog-breeds/american-bulldog',
 'https://dogtime.com/dog-breeds/shiba-inu',
 'https://dogtime.com/dog-breeds/cairn-terrier',
 'https://dogtime.com/dog-breeds/weimaraner',
 'https://dogtime.com

In [14]:
executable_path = {'executable_path': 'chromedriver.exe'}


In [15]:
#intialize dataframe row lists
#source: https://stackoverflow.com/questions/10715965/add-one-row-to-pandas-dataframe
rows_list = []
#open browser
browser = Browser('chrome', **executable_path, headless=False)

#visit each link and extract rating data
for dog_link in search_list:
    #initialize list
    stars_list = []
    # Visit each dogtime url
    browser.visit(dog_link)
    
     # Scrape page into Soup
    html2 = browser.html
    soup2 = bs(html2, "lxml")
    
    dog_name_finder = soup2.find_all('div', class_= "breeds-single-content")[0]
    #find breed and append to list
    breed = dog_name_finder.find('h1').text
    stars_list.append(breed)
    
    #find area of page with details about star ratings
    check = soup2.find_all('div', class_= "characteristic-stars parent-characteristic")

    #iterate through each category and get rating
    for dog_type in check:
        #find star rating
        star = str(dog_type.find('div'))
        #add star rating to list
        stars_list.append(star)
    #add breed/star list to dataframe rows list
    rows_list.append(stars_list)

    

In [16]:
#create dataframe with all row lists
dog_trait_df = pd.DataFrame(rows_list)   

In [17]:
dog_trait_df

,0,1,2,3,4,5
0,Greyhound,"<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-5""></div></div>","<div class=""characteristic-star-block""><div class=""star star-5""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>"
1,Pomeranian,"<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-3""></div></div>"
2,Akita,"<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-2""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>"
3,Bloodhound,"<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-5""></div></div>","<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>"
4,Collie,"<div class=""characteristic-star-block""><div class=""star star-4""></div></div>","<div class=""characteristic-star-block""><div class=""star star-5""></div></div>","<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-3""></div></div>"
5,Black and Tan Coonhound,"<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-5""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>","<div class=""characteristic-star-block""><div class=""star star-5""></div></div>"
6,Pekingese,"<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-2""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>","<div class=""characteristic-star-block""><div class=""star star-2""></div></div>"
7,Rottweiler,"<div class=""characteristic-star-block""><div class=""star star-2""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>"
8,Dutch Shepherd,"<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>","<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteristic-star-block""><div class=""star star-4""></div></div>"
9,Rhodesian Ridgeback,"<div class=""characteristic-star-block""><div class=""star star-3""></div></div>","<div class=""characteri

In [18]:
#clean up ratings columns
#souce: https://datatofish.com/replace-values-pandas-dataframe/
#source: https://thispointer.com/pandas-loop-or-iterate-over-all-or-certain-columns-of-a-dataframe/#:~:text=iteritems()%20i.e.-,DataFrame.iteritems(),and%20column%20contents%20as%20series.&text=As%20there%20were%203%20columns%20so%203%20tuples%20were%20returned%20during%20iteration.
for column in dog_trait_df:
    dog_trait_df[column] = dog_trait_df[column].replace(['<div class="characteristic-star-block"><div class="star star-1"></div></div>'],1)
    dog_trait_df[column] = dog_trait_df[column].replace(['<div class="characteristic-star-block"><div class="star star-2"></div></div>'],2)
    dog_trait_df[column] = dog_trait_df[column].replace(['<div class="characteristic-star-block"><div class="star star-3"></div></div>'],3)
    dog_trait_df[column] = dog_trait_df[column].replace(['<div class="characteristic-star-block"><div class="star star-4"></div></div>'],4)
    dog_trait_df[column] = dog_trait_df[column].replace(['<div class="characteristic-star-block"><div class="star star-5"></div></div>'],5)

In [19]:
#rename columns
dog_trait_df = dog_trait_df.rename(columns = {0:"Breed",
                                             1:"Adaptability",
                                             2:"All Around Friendliness",
                                             3:"Health and Grooming Needs",
                                             4:"Trainability",
                                             5:"Physical Needs"})

In [20]:
#set breed as index
dog_trait_df.set_index('Breed', inplace = True)

In [21]:
dog_trait_df

,Adaptability,All Around Friendliness,Health and Grooming Needs,Trainability,Physical Needs
Breed,,,,,
Greyhound,3,5,5,4,4
Pomeranian,3,3,3,3,3
Akita,3,2,4,4,4
Bloodhound,3,5,3,4,4
Collie,4,5,3,3,3
Black and Tan Coonhound,3,5,4,4,5
Pekingese,3,3,2,4,2
Rottweiler,2,4,4,4,4
Dutch Shepherd,3,4,3,3,4
